# Imports und Definitionen

In [1]:
import glob
import json
import os
import datetime

import pandas as pd
import numpy as np
import plotly.express as px

pd.set_option('display.max_rows', 100)

In [2]:
DATA_PATH = "../vwkommi"
json_files = glob.glob(f"{DATA_PATH}/*.json")

g_sheet_id = "16qU2KyGPyNVdnJiGuG-ALUxbVxHwq9gv"
g_sheet_name = "Liste"
g_sheet_url = f"https://docs.google.com/spreadsheets/d/{g_sheet_id}/gviz/tq?tqx=out:csv&sheet={g_sheet_name}"
g_sheet_header = [
    "Bestelldatum",
    "Komm.Nr.",
    "FIN vorhanden",
    "Modell",
    "Modelljahr",
    "Variante",
    "Farbe",
    "Assistenz-Paket",
    "Designpaket",
    "Infotainment",
    "Komfort-Paket",
    "Sport Paket",
    "WP",
    "AH",
    "Geliefert"
]

g_sheet_months = {
    'Jan.': "January",
    'Feb.': "February",
    'März': "March",
    'Apr.': "April",
    'Mai': "May",
    'Juni': "June",
    'Juli': "July",
    'Aug.': "August",
    'Sept.': "September",
    'Okt.': "October",
    'Nov.': "November",
    'Dez.': "December",
}


In [3]:
def load_file(path):
    file_name = os.path.basename(os.path.splitext(path)[0])
    date = file_name.split("_")[-1].split("T")[0]

    cars = []

    with open(path, "r") as f:
        json_string = json.loads(f.read())

    for komm_num, elems in json_string.items():
        car = {
            "Datum": datetime.datetime.strptime(date, "%Y-%m-%d"),
            "Kommissionsnummer": komm_num,
            "Kommissionsbereich": komm_num[:2],
            "FIN": "",
            "Modell": "",
            "Farbcode": "",
            "Farbe": "",
            "Motor": "",
            "Modelljahr": "",
            "Ausstattung": "",
            "Softwareverbund": 0,
            "Assistenzpaket Plus": False,
            "Memory-Sitze": False,
            "Fertigungsablauf": "Standard"
        }

        num_elems = len(elems)
        for num, elem in zip(range(num_elems), elems):
            if num == 0:
                try:
                    car['FIN'] = elem["vin"]
                except KeyError:
                    car['FIN'] = ""
                
                car['Modell'] = elem["modelName"].split(" ")[0]
                car['Variante'] = ' '.join(elem["modelName"].split(" ")[1:])
                car['Farbcode'] = elem["exteriorColor"]

            if num == 1:
                car['Motor']  = elem["engine"]
                car['Modelljahr']  = elem["modelYear"]
                try:
                    car['Farbe'] = elem["exteriorColorText"]
                except KeyError:
                    car['Farbe'] = ""

                try:
                    specifications = elem["specifications"]
                    
                    specs = []
                    for spec in specifications:
                        specs.append(spec["codeText"])

                        if "Softwareverbund" in spec["codeText"]:
                            try:
                                car["Softwareverbund"] = int(spec["codeText"].split(" ")[-1])
                            except ValueError:
                                pass

                        if 'Umgebungsansicht "Area View" inkl. Rückfahrkamera "Rear View"' in spec["codeText"]:
                            car["Assistenzpaket Plus"] = True

                        if 'ergoActive-Sitze vorn mit elektrischer Einstellung, Memory-Funktion und verschiebbarer Oberschenkelauflage' in spec["codeText"]:
                            car["Memory-Sitze"] = True

                        if "Fertigungsablauf" in spec["codeText"]:
                            ort = spec["codeText"].split(" ")[-1]

                            if ort == "Dresden":
                                ort = "Standard"
                                
                            car["Fertigungsablauf"] = ort

                    car['Ausstattung'] = specs

                except KeyError:
                    car['Ausstattung'] = ""

        cars.append(car)

    df = pd.DataFrame(cars)

    return df


In [4]:
def clean_date(date):
    month, year = date.split("/")

    return pd.to_datetime(f"{g_sheet_months[month]}/{year.strip()}", format="%B/%y")

# Load Data

In [5]:
dfs = []
num_files = len(json_files)
for file, num in zip(json_files, range(len(json_files))):
    print(f"Parsing {file} ({num}/{num_files})")
    _df = load_file(file)
    dfs.append(_df)

_df = pd.concat(dfs)
df = _df[(_df["Modell"] != "") & (_df["Variante"] != "") & ~(_df["Variante"].isna())]

Parsing ../vwkommi/output_AL_0-9999_2022-09-03T16.34.56.json (0/152)
Parsing ../vwkommi/output_AK_0-9999_2022-08-28T16.46.01.json (1/152)
Parsing ../vwkommi/output_AJ_0-9999_2022-04-30T11.13.34.json (2/152)
Parsing ../vwkommi/output_AF_5000-9999_2022-04-09T10.57.49.json (3/152)
Parsing ../vwkommi/output_AL_0-9999_2022-09-10T11.45.20.json (4/152)
Parsing ../vwkommi/output_AF_5000-9999_2022-09-17T09.48.15.json (5/152)
Parsing ../vwkommi/output_AH_0-9999_2022-06-14T18.51.25.json (6/152)
Parsing ../vwkommi/output_AF_5000-9999_2022-03-29T17.44.56.json (7/152)
Parsing ../vwkommi/output_AJ_0-9999_2022-04-23T10.54.17.json (8/152)
Parsing ../vwkommi/output_AI_0-9999_2022-05-15T10.52.21.json (9/152)
Parsing ../vwkommi/output_AJ_0-9999_2022-05-07T09.48.18.json (10/152)
Parsing ../vwkommi/output_AI_0-9999_2022-09-10T11.45.20.json (11/152)
Parsing ../vwkommi/output_AI_0-9999_2022-09-03T16.34.56.json (12/152)
Parsing ../vwkommi/output_AH_0-9999_2022-06-01T19.11.16.json (13/152)
Parsing ../vwkommi/ou

In [6]:
g_sheet_df = pd.read_csv(g_sheet_url, names=g_sheet_header, skiprows=1, usecols=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])
g_sheet_df.dropna(axis=0, subset="Bestelldatum", inplace=True)

g_sheet_df["Komm.Nr."] = g_sheet_df["Komm.Nr."].str[:2]
g_sheet_df["Bestelldatum"] = g_sheet_df["Bestelldatum"].apply(clean_date)
g_sheet_df = g_sheet_df.replace({'FIN vorhanden': {'Ja': True, 'Nein': False}})
g_sheet_df = g_sheet_df.replace({'Assistenz-Paket': {'Ja': True, 'Nein': False}})
g_sheet_df = g_sheet_df.replace({'Designpaket': {'Ja': True, 'Nein': False, "k.A.": np.NaN}})
g_sheet_df = g_sheet_df.replace({'Infotainment': {'Ja': True, 'Nein': False, "k.A.": np.NaN}})
g_sheet_df = g_sheet_df.replace({'Komfort-Paket': {'Ja': True, 'Nein': False, "k.A.": np.NaN}})
g_sheet_df = g_sheet_df.replace({'Sport Paket': {'Ja': True, 'Nein': False, "k.A.": np.NaN}})
g_sheet_df = g_sheet_df.replace({'WP': {'Ja': True, 'Nein': False, "k.A.": np.NaN}})
g_sheet_df = g_sheet_df.replace({'AH': {'Ja': True, 'Nein': False, "k.A.": np.NaN}})
g_sheet_df = g_sheet_df.replace({'Geliefert': {'Ja': True, 'Nein': False}})
g_sheet_df = g_sheet_df.replace({'Modelljahr': {'Modelljahr': np.NaN}})
g_sheet_df = g_sheet_df.replace({'Farbe': {'Bitte wählen': np.NaN, "Sonstige": np.NaN}})

g_sheet_df.rename(columns={
    "Komm.Nr.": "Kommissionsbereich",
    "FIN vorhanden": "FIN",
    "Assistenz-Paket": "Assistenzpaket",
    "Infotainment": "Infotainmentpaket",
    "Komfort-Paket": "Komfortpaket",
    "Sport Paket": "Sportpaket",
    "WP": "Wärmepumpe",
    "AH": "Anhängerkupplung",
    "Geliefert": "Status"
    }, inplace=True)

In [7]:
g_sheet_df.head()

,Bestelldatum,Kommissionsbereich,FIN,Modell,Modelljahr,Variante,Farbe,Assistenzpaket,Designpaket,Infotainmentpaket,Komfortpaket,Sportpaket,Wärmepumpe,Anhängerkupplung,Status
0,2021-05-01,AF,True,ID.4,2022,Pro Performance,NaN,Plus,Plus,Plus,Plus,Plus,True,True,True
1,2021-05-01,AF,False,ID.4,NaN,GTX,Kings Red,Plus,Plus,Plus,Plus,Plus,False,True,False
2,2021-05-01,AF,True,ID.4,2022,Pro Performance,Kings Red,Plus,Plus,Plus,Plus,Plus,True,True,True
3,2021-05-01,AF,True,ID.4,NaN,Pro Performance,Blue Dusk,Plus,Plus,Plus,Plus,Plus,True,True,In Produktion
4,2021-05-01,AF,True,ID.4,2022,GTX,Mangangrau,Plus,Plus,Plus,Plus,Plus,True,False,True


# VW Kommi Visualisierungen

In [8]:
model_df = df[['Datum', 'Modell', 'Kommissionsnummer']].groupby(['Datum', 'Modell']).nunique().reset_index()

In [9]:
fig = px.line(
    model_df,
    x="Datum",
    y="Kommissionsnummer",
    color="Modell",
    #title='Anzahl Fahrzeuge Pro Woche nach Modell',
    markers=True,
    labels={
        "Kommissionsnummer": "Anzahl",
    }
)

fig.write_html("../html/images/model_count.html", include_plotlyjs="cdn", full_html=False)

In [10]:
varianten_df = df[['Datum', 'Modell', "Variante", 'Kommissionsnummer']].groupby(['Datum', 'Modell', "Variante"]).nunique().reset_index()

In [11]:
fig = px.line(
    varianten_df,
    x="Datum",
    y="Kommissionsnummer",
    color="Variante",
    #title='Anzahl Fahrzeuge Pro Wochenach Variante',
    facet_col="Modell",
    markers=True,
    labels={
        "Kommissionsnummer": "Anzahl",
    }
)

fig.write_html("../html/images/variants_count.html", include_plotlyjs="cdn", full_html=False)

In [12]:
kommissionsbereich_df = df[['Datum', 'Modell', "Kommissionsbereich", 'Kommissionsnummer']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [13]:
fig = px.line(
    kommissionsbereich_df,
    x="Datum",
    y="Kommissionsnummer",
    color="Kommissionsbereich",
    #title='Anzahl Fahrzeuge Pro Woche nach Kommisionsnummerbereich',
    facet_col="Modell",
    markers=True,
    labels={
        "Kommissionsnummer": "Anzahl",
    }
)

fig.write_html("../html/images/kommbereich_count.html", include_plotlyjs="cdn", full_html=False)

In [14]:
fin_df = df[['Datum', 'Modell', 'FIN']].groupby(['Datum', 'Modell']).nunique().reset_index()

In [15]:
fig = px.line(
    fin_df,
    x="Datum",
    y="FIN",
    color="Modell",
    #title='Anzahl FINs Pro Woche nach Modell',
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/fin_count.html", include_plotlyjs="cdn", full_html=False)

In [16]:
fin_varianten_df = df[['Datum', 'Modell', "Variante", 'FIN']].groupby(['Datum', 'Modell', "Variante"]).nunique().reset_index()

In [17]:
fig = px.line(
    fin_varianten_df,
    x="Datum",
    y="FIN",
    color="Variante",
    #title='Anzahl FINs Pro Woche nach Variante',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/fin_variants_count.html", include_plotlyjs="cdn", full_html=False)

In [18]:
bereich_varianten_df = df[['Datum', 'Modell', "Kommissionsbereich", 'FIN']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [19]:
fig = px.line(
    bereich_varianten_df,
    x="Datum",
    y="FIN",
    color="Kommissionsbereich",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/komm_range_count.html", include_plotlyjs="cdn", full_html=False)

In [20]:
assi_plus_df = df[df["Assistenzpaket Plus"] == True]
memory_df = df[df["Memory-Sitze"] == True]
both_df = df[(df["Assistenzpaket Plus"] == True) & (df["Memory-Sitze"] == True)]
none_df = df[(df["Assistenzpaket Plus"] == False) & (df["Memory-Sitze"] == False)]

In [21]:
assi_plus_df_df = assi_plus_df[['Datum', 'Modell', "Kommissionsbereich", 'FIN']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [22]:
fig = px.line(
    assi_plus_df_df,
    x="Datum",
    y="FIN",
    color="Kommissionsbereich",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich für Autos mit Assistenzpaket "Plus" (ohne Memory-Sitze)',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/assiplus_count.html", include_plotlyjs="cdn", full_html=False)

In [23]:
memory_df_df = memory_df[['Datum', 'Modell', "Kommissionsbereich", 'FIN']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [24]:
fig = px.line(
    memory_df_df,
    x="Datum",
    y="FIN",
    color="Kommissionsbereich",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich für Autos mit Memory Sitzen (ohne Assistenspaket "Plus")',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/memory_count.html", include_plotlyjs="cdn",full_html=False)

In [25]:
both_df_df = both_df[['Datum', 'Modell', "Kommissionsbereich", 'FIN']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [26]:
fig = px.line(
    both_df_df,
    x="Datum",
    y="FIN",
    color="Kommissionsbereich",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich für Autos mit Assistenzpaket "Plus" und Memory Sitzen',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/both_count.html", include_plotlyjs="cdn", full_html=False)

In [27]:
none_df_df = none_df[['Datum', 'Modell', "Kommissionsbereich", 'FIN']].groupby(['Datum', 'Modell', "Kommissionsbereich"]).nunique().reset_index()

In [28]:
fig = px.line(
    none_df_df,
    x="Datum",
    y="FIN",
    color="Kommissionsbereich",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich für Autos ohne Assistenzpaket "Plus" und ohne Memory Sitze',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/none_count.html", include_plotlyjs="cdn", full_html=False)

In [29]:
swv_df = df[df["Softwareverbund"].between(6, 10)][['Datum', 'Modell', "Softwareverbund", 'FIN']].groupby(['Datum', 'Modell', "Softwareverbund"]).nunique().reset_index()

In [30]:
fig = px.line(
    swv_df,
    x="Datum",
    y="FIN",
    color="Softwareverbund",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/software.html", include_plotlyjs="cdn", full_html=False)

In [31]:
fertigung_df = df[['Datum', 'Modell', "Fertigungsablauf", 'FIN']].groupby(['Datum', 'Modell', "Fertigungsablauf"]).nunique().reset_index()

In [32]:
fig = px.line(
    fertigung_df,
    x="Datum",
    y="FIN",
    color="Fertigungsablauf",
    #title='Anzahl FINs Pro Woche nach Kommisionsnummerbereich',
    facet_col="Modell",
    markers=True,
    labels={
        "FIN": "Anzahl",
    }
)

fig.write_html("../html/images/fertigung.html", include_plotlyjs="cdn", full_html=False)

# Google Sheet Visualisierungen